In [ ]:
import numpy as np
import scipy.special
import pandas ad pd

# Define parameters
N = 5  # Number of machines
lambda_machine = 1/8  # Breakdown rate per machine (1 failure per 8 hours)
mu = 1/2  # Repair rate per repairman (1 repair per 2 hours)
cost_repairman_per_hour = 10  # Cost per repairman per hour ($)
cost_downtime_per_machine_per_hour = 50  # Cost per machine down per hour ($)

# Function to compute probability of zero machines in the system (P0)
def compute_P0(N, lambda_machine, mu, c):
    lambda_total = N * lambda_machine  # Total breakdown arrival rate
    rho = lambda_total / mu  # Utilization factor

    sum1 = sum([(scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(n))) *
                (rho ** n) for n in range(c)])
    sum2 = sum([(scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(c) * (c ** (n - c)))) *
                (rho ** n) for n in range(c, N + 1)])

    P0 = 1 / (sum1 + sum2)
    return P0

# Function to compute expected number of machines in the system (L)
def compute_L(N, lambda_machine, mu, c):
    P0 = compute_P0(N, lambda_machine, mu, c)
    lambda_total = N * lambda_machine  # Total breakdown arrival rate
    rho = lambda_total / mu  # Utilization factor

    sum1 = sum([(scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(n))) *
                (rho ** n) * P0 * n for n in range(c)])
    sum2 = sum([(scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(c) * (c ** (n - c)))) *
                (rho ** n) * P0 * n for n in range(c, N + 1)])

    L = sum1 + sum2
    return L

# Compute costs for different values of c
results = []
for c in range(1, 6):  # Evaluating c = 1 to 5
    L = compute_L(N, lambda_machine, mu, c)  # Expected number of machines in the system
    total_cost = (cost_repairman_per_hour * c) + (cost_downtime_per_machine_per_hour * L)  # Total expected cost
    results.append((c, L, total_cost))

# Convert results to a DataFrame and display
import pandas as pd
df_results = pd.DataFrame(results, columns=["Number of Repairmen (c)", "Expected Machines Down (L)", "Total Cost per Hour ($)"])

display(df_results)

# Extract the row with the smallest total cost per hour
min_cost_row = df_results.loc[df_results["Total Cost per Hour ($)"].idxmin()]

# Display the result
print("Optimal Number of Repairmen and Cost:")
print(min_cost_row)



In [14]:
import numpy as np
import scipy.special
import pandas as pd

# Given parameters from the examtext

N = 5 # Number of machines
lambda_m = 1/8 # Failure rate per machine
mu = 1/2 # repair rate per repairman

# Function to compute P0, Pn, L, Lq, W, Wq, and cost for different values for c
def compute_queuing_metrics(N,lambda_m,mu,c_values):
  results = []
  for c in c_values:
    lambda_n = [(N-n) * lambda_m for n in range(N +1)]
    mu_n = [min(n,c)* mu for n in range(N+1)]

    # Compute P0 using the formula

    formula_part_1 = sum([scipy.special.fratorial(N)/(scipy.special.factorial(N-n)*scipy.special.factorial(n))*
                         (lambda_m / mu) ** n for n in range(c)])

    formula_part_2 = sum([scipy.special.fratorial(N)/(scipy.special.factorial(N-n)*scipy.special.factorial(c) * c**(n-c))*
                          (lambda_m/mu) ** n for n in range(c, N+1)])

    P0 = 1/ (formula_part_1 + formula_part_2)

    # Compute Pn for n = 0 to N
    Pn = [P0 * (scipy.special.factorial(N) / (scipy.special.factorial(N-n) * scipy.sparse.factorial(n)))*
          (lambda_m / mu) ** n if n < c else
          P0 * (scipy.special.factorial(N)/(scipy.special.factorial(N-n)*scipy.special.factorial(c) * c**(n-c)))*
          (lambda_m/mu)** n for n in range(N+1)]

    # Compute L (Mean number of broken machines in the system)

    L = sum(n*Pn[n] for n in range(N+1))

    # Compute Lq (Mean number of machines in queue)
    Lq = sum((n-c) * Pn[n] for i in range(c, N+1))

    # Compute W and Wq using Little's Law
    lambda_effective = lambda_m * (N - L)
    W = L / lambda_effective if lambda_effective > 0 else 0
    Wq = Lq / lambda_effective if lambda_effective > 0 else 0

    # cost computition

    downtime_cost = 50 * L
    repairmen_cost = 10 * c
    total_cost = downtime_cost + repairmen_cost

    results.append([c, L, Lq, W, Wq, downtime_cost, repairmen_cost, total_cost])

  return resuslts

  # evaluate for c = 1...5

  c_values = range(1,6)
  queuing_results = compute_queuing_metrics(N, lambda_m, mu, c_values)

  # dataframe
  df_results = pd.DataFrame(queuing_results, columns = ["c", "L", "Lq", "W", "Wq", "DowntimeCost", "RepairmenCost", "TotalCost" ])

  display(df_results)


In [13]:
import numpy as np
import scipy.special

# Given parameters
N = 5  # Number of machines
lambda_m = 1/8  # Failure rate per machine
mu = 1/2  # Repair rate per repairman

# Define a function to compute P0, Pn, L, Lq, W, Wq, and cost for different values of c
def compute_queuing_metrics(N, lambda_m, mu, c_values):
    results = []
    for c in c_values:
        # Compute λ_n and μ_n for different n states
        lambda_n = [(N - n) * lambda_m for n in range(N + 1)]
        mu_n = [min(n, c) * mu for n in range(N + 1)]

        # Compute P0 using the given formula
        sum_part_1 = sum([scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(n)) *
                           (lambda_m / mu) ** n for n in range(c)])

        sum_part_2 = sum([scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(c) * c**(n - c)) *
                           (lambda_m / mu) ** n for n in range(c, N + 1)])

        P0 = 1 / (sum_part_1 + sum_part_2)

        # Compute Pn for n = 0 to N
        Pn = [P0 * (scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(n))) *
              (lambda_m / mu) ** n if n < c else
              P0 * (scipy.special.factorial(N) / (scipy.special.factorial(N - n) * scipy.special.factorial(c) * c**(n - c))) *
              (lambda_m / mu) ** n for n in range(N + 1)]

        # Compute L (Mean number of broken machines in the system)
        L = sum(n * Pn[n] for n in range(N + 1))

        # Compute Lq (Mean number of machines in queue)
        Lq = sum((n - c) * Pn[n] for n in range(c, N + 1))

        # Compute W and Wq using Little’s Law
        lambda_effective = lambda_m * (N - L)  # Effective arrival rate
        W = L / lambda_effective if lambda_effective > 0 else 0
        Wq = Lq / lambda_effective if lambda_effective > 0 else 0

        # Compute costs
        downtime_cost = 50 * L  # Cost of broken machines
        repairmen_cost = 10 * c  # Cost of repairmen
        total_cost = downtime_cost + repairmen_cost  # Total cost

        results.append([c, L, Lq, W, Wq, downtime_cost, repairmen_cost, total_cost])

    return results

# Evaluate for c = 1 to 5
c_values = range(1, 6)
queuing_results = compute_queuing_metrics(N, lambda_m, mu, c_values)

# Convert results to DataFrame for better visualization
import pandas as pd
df_results = pd.DataFrame(queuing_results, columns=["c", "L", "Lq", "W", "Wq", "Downtime Cost", "Repairmen Cost", "Total Cost"])

# Display results

display(df_results)


,c,L,Lq,W,Wq,Downtime Cost,Repairmen Cost,Total Cost
0,1,1.796267,0.995334,4.485437,2.485437,89.813375,10,99.813375
1,2,1.094110,0.117638,2.240945,0.240945,54.705521,20,74.705521
2,3,1.007944,0.009930,2.019900,0.019900,50.397219,30,80.397219
3,4,1.000320,0.000400,2.000800,0.000800,50.015999,40,90.015999
4,5,1.000000,0.000000,2.000000,0.000000,50.000000,50,100.000000
